# hello_milvus Demo

`hello_milvus.ipynb` demonstrates the basic operations of PyMilvus, a Python SDK of Milvus.
Before running, make sure that you have a running Milvus instance.

1. connect to Milvus
2. create collection
3. insert data
4. create index
5. search, query, and hybrid search on entities
6. delete entities by PK
7. drop collection

In [1]:
import numpy as np
import time

from pymilvus import (
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)

fmt = "\n=== {:30} ===\n"
search_latency_fmt = "search latency = {:.4f}s"
num_entities, dim = 3000, 8

## 1. connect to Milvus

Add a new connection alias `default` for Milvus server in `localhost:19530`. 

Actually the `default` alias is a buildin in PyMilvus. If the address of Milvus is the same as `localhost:19530`, you can omit all parameters and call the method as: `connections.connect()`.

Note: the `using` parameter of the following methods is default to "default".

In [3]:
connections.connect("default", host="localhost", port="19530")

has = utility.has_collection("hello_milvus")
print(f"Does collection hello_milvus exist in Milvus: {has}")

Does collection hello_milvus exist in Milvus: True


## 2. create collection
We're going to create a collection with 3 fields.

|   |field name  |field type |other attributes              |  field description      |
|---|:----------:|:---------:|:----------------------------:|:-----------------------:|
|1  |    "pk"    |   VARCHAR |is_primary=True, auto_id=False|      "primary field"    |
|2  |  "random"  |   Double  |                              |      "a double field"   |
|3  |"embeddings"|FloatVector|     dim=8                    |"float vector with dim 8"|

In [4]:
fields = [
    FieldSchema(name="pk", dtype=DataType.VARCHAR, is_primary=True, auto_id=False, max_length=100),
    FieldSchema(name="random", dtype=DataType.DOUBLE),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=dim)
]

schema = CollectionSchema(fields, "hello_milvus is the simplest demo to introduce the APIs")

hello_milvus = Collection("hello_milvus", schema, consistency_level="Strong")

## 3. insert data

We are going to insert 3000 rows of data into `hello_milvus`. Data to be inserted must be organized in fields.

The insert() method returns:
- either automatically generated primary keys by Milvus if auto_id=True in the schema;
- or the existing primary key field from the entities if auto_id=False in the schema.

In [5]:
rng = np.random.default_rng(seed=19530)
entities = [
    # provide the pk field because `auto_id` is set to False
    [str(i) for i in range(num_entities)],
    rng.random(num_entities).tolist(),  # field random, only supports list
    rng.random((num_entities, dim)),    # field embeddings, supports numpy.ndarray and list
]

insert_result = hello_milvus.insert(entities)

print(f"Number of entities in Milvus: {hello_milvus.num_entities}")  # check the num_entites

Number of entities in Milvus: 15000


## 4. create index
We are going to create an IVF_FLAT index for hello_milvus collection.

create_index() can only be applied to `FloatVector` and `BinaryVector` fields.

In [6]:
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}

hello_milvus.create_index("embeddings", index)

Status(code=0, message=)

## 5. search, query, and hybrid search
After data were inserted into Milvus and indexed, you can perform:
- search based on vector similarity
- query based on scalar filtering(boolean, int, etc.)
- hybrid search based on vector similarity and scalar filtering.

Before conducting a search or a query, you need to load the data in `hello_milvus` into memory.

In [7]:
hello_milvus.load()

**Search based on vector similarity**

In [8]:
vectors_to_search = entities[-1][-2:]
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 10},
}

start_time = time.time()
result = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=3, output_fields=["random"])
end_time = time.time()

for hits in result:
    for hit in hits:
        print(f"hit: {hit}, random field: {hit.entity.get('random')}")
print(search_latency_fmt.format(end_time - start_time))

hit: id: 2998, distance: 0.0, entity: {'random': 0.9728033590489911}, random field: 0.9728033590489911
hit: id: 1262, distance: 0.08883658051490784, entity: {'random': 0.2978858685751561}, random field: 0.2978858685751561
hit: id: 1265, distance: 0.09590047597885132, entity: {'random': 0.3042039939240304}, random field: 0.3042039939240304
hit: id: 2999, distance: 0.0, entity: {'random': 0.02316334456872482}, random field: 0.02316334456872482
hit: id: 1580, distance: 0.05628091096878052, entity: {'random': 0.3855988746044062}, random field: 0.3855988746044062
hit: id: 2377, distance: 0.08096685260534286, entity: {'random': 0.8745922204004368}, random field: 0.8745922204004368
search latency = 0.1654s


In [9]:
result

['["id: 2998, distance: 0.0, entity: {\'random\': 0.9728033590489911}", "id: 1262, distance: 0.08883658051490784, entity: {\'random\': 0.2978858685751561}", "id: 1265, distance: 0.09590047597885132, entity: {\'random\': 0.3042039939240304}"]', '["id: 2999, distance: 0.0, entity: {\'random\': 0.02316334456872482}", "id: 1580, distance: 0.05628091096878052, entity: {\'random\': 0.3855988746044062}", "id: 2377, distance: 0.08096685260534286, entity: {\'random\': 0.8745922204004368}"]']

In [10]:
result[0]

["id: 2998, distance: 0.0, entity: {'random': 0.9728033590489911}", "id: 1262, distance: 0.08883658051490784, entity: {'random': 0.2978858685751561}", "id: 1265, distance: 0.09590047597885132, entity: {'random': 0.3042039939240304}"]

In [25]:
[item.id for item in result[0]], [item.distance for item in result[0]], [item.entity.random for item in result[0]], [item.fields for item in result[0]]

(['2998', '1262', '1265'],
 [0.0, 0.08883658051490784, 0.09590047597885132],
 [0.9728033590489911, 0.2978858685751561, 0.3042039939240304],
 [{'random': 0.9728033590489911},
  {'random': 0.2978858685751561},
  {'random': 0.3042039939240304}])

In [20]:
result[0].ids, result[0].distances

(['2998', '1262', '1265'], [0.0, 0.08883658051490784, 0.09590047597885132])

In [26]:
f"id in [{','.join([97, 3])}]"

TypeError: sequence item 0: expected str instance, int found

**Query based on scalar filtering(boolean, int, etc.)**

Start quering with `random > 0.5`

In [10]:
start_time = time.time()
result = hello_milvus.query(expr="random > 0.5", output_fields=["random", "embeddings"])
end_time = time.time()

print(f"query result:\n-{result[0]}")
print(search_latency_fmt.format(end_time - start_time))

query result:
-{'random': 0.6378742006852851, 'embeddings': [0.20963514, 0.39746657, 0.12019053, 0.6947492, 0.9535575, 0.5454552, 0.82360446, 0.21096309], 'pk': '0'}
search latency = 0.2621s


**Hybrid search**

Start hybrid searching with `random > 0.5`

In [11]:
start_time = time.time()
result = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=3, expr="random > 0.5", output_fields=["random"])
end_time = time.time()

for hits in result:
    for hit in hits:
        print(f"hit: {hit}, random field: {hit.entity.get('random')}")
print(search_latency_fmt.format(end_time - start_time))

hit: id: 2998, distance: 0.0, entity: {'random': 0.9728033590489911}, random field: 0.9728033590489911
hit: id: 747, distance: 0.14606499671936035, entity: {'random': 0.5648774800635661}, random field: 0.5648774800635661
hit: id: 2527, distance: 0.1530652642250061, entity: {'random': 0.8928974315571507}, random field: 0.8928974315571507
hit: id: 2377, distance: 0.08096685260534286, entity: {'random': 0.8745922204004368}, random field: 0.8745922204004368
hit: id: 2034, distance: 0.20354536175727844, entity: {'random': 0.5526117606328499}, random field: 0.5526117606328499
hit: id: 958, distance: 0.21908017992973328, entity: {'random': 0.6647383716417955}, random field: 0.6647383716417955
search latency = 0.2104s


## 6. delete entities by PK
You can delete entities by their PK values using boolean expressions.


In [12]:
ids = insert_result.primary_keys
expr = f'pk in ["{ids[0]}", "{ids[1]}"]'

result = hello_milvus.query(expr=expr, output_fields=["random", "embeddings"])
print(f"query before delete by expr=`{expr}` -> result: \n-{result[0]}\n-{result[1]}\n")

hello_milvus.delete(expr)

result = hello_milvus.query(expr=expr, output_fields=["random", "embeddings"])
print(f"query after delete by expr=`{expr}` -> result: {result}\n")

query before delete by expr=`pk in ["0", "1"]` -> result: 
-{'random': 0.6378742006852851, 'embeddings': [0.20963514, 0.39746657, 0.12019053, 0.6947492, 0.9535575, 0.5454552, 0.82360446, 0.21096309], 'pk': '0'}
-{'random': 0.43925103574669633, 'embeddings': [0.52323616, 0.8035404, 0.77824664, 0.80369574, 0.4914803, 0.8265614, 0.6145269, 0.80234545], 'pk': '1'}

query after delete by expr=`pk in ["0", "1"]` -> result: []



## 7. drop collection
Finally, drop the hello_milvus collection

In [13]:
utility.drop_collection("hello_milvus")